### **1. Install and Import Libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
! pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


### **2. Load the test data**

In [5]:
from datasets import load_from_disk
tokenized_dataset_test = load_from_disk("/content/drive/MyDrive/wikihow_data/T5small_test_tokenized")

In [6]:
import pandas as pd
dataset = pd.DataFrame()

In [7]:
dataset['text'] = tokenized_dataset_test['text']
dataset['headline'] = tokenized_dataset_test['headline']

In [10]:
dataset

,text,headline
0,"Caged animals such as hamsters, guinea pigs, r...",Relocate your caged pet.\nChoose an environmen...
1,Every boarding barn has people who go on vacat...,Horse-sit.\nHire yourself out as a groomer.\nT...
2,Let him know that you like him by making an ef...,Pay attention to your appearance.\nShow him ho...
3,It can be hard to tell if someone is intereste...,Read the signs.\nInvite him along on a group a...
4,Your crush may be the prettiest person in the ...,Get to know your crush a bit.\nHang out in per...
...,...,...
5572,";\n, If you're having trouble with the seatbel...",Find someone you feel comfortable with to go o...
5573,Listen for the click or other sound it makes w...,"Double check your harness, bar or other restra..."
5574,Each class corresponds to a letter of the alph...,Learn the 20 classifications the Library of Co...
5575,Brake pads and brake shoes are different from ...,Learn the difference between brake pads and br...


In [22]:
dataset['text'][0]

"Caged animals such as hamsters, guinea pigs, rodent, reptiles, and amphibians can be taken to a friend's or sitter's home. Create a document that outlines the feeding and water needs, cleaning schedule, and temperature control. Pack all of the things that mimic your pet's environment at your home such as bedding, heated surfaces, and decorations.If the cage is not mobile, someone will need to come check on your pet daily.\n A rabbit, ferret, or guinea pig is live bait in the wild. Relocating your pet to a home with canines or young children can be stressful and dangerous for your pet. If your pet is used to being in a quiet home with only adults, find a place that mimics that.The relocation environment is extremely important. Your pet can become disoriented and possibly sick from sudden lifestyle changes.\n Birds and cats enjoy being in consistent, familiar environments. Birds may become restless and pluck their own feathers in unfamiliar environments. Similarly, cats can become distr

In [24]:
dataset['headline'][0]

'Relocate your caged pet.\nChoose an environment that is similar to your home.\nGet an in-home sitter for birds and cats.\nTake your pet to a kennel.'

In [ ]:
X_test, y_test = dataset['text'], dataset['headline']

### **3. Define Metrics Utility Functions**

In [8]:
# Set the locale to UTF-8
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=63489e12c7a59a4f5ab8159104a5826d50f003c2c3fc5054a1c629232a367e5a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [16]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

def getmetrics(dataset,model_summaries):
  # Sample references and hypotheses
  references = [ref.split() for ref in dataset['headline']]

  if model_summaries in ['T5_SFT_summary', 'T5_LoRA_summary']:
    hypotheses = [hyp[0]['summary_text'].split() for hyp in dataset[model_summaries]]
  else:
    hypotheses = [hyp.split() for hyp in dataset[model_summaries]]

# Calculate BLEU score
  bleu_scores = [sentence_bleu([ref], hyp) for ref, hyp in zip(references, hypotheses)]

# Calculate average BLEU score
  average_bleu = sum(bleu_scores) / len(bleu_scores)

  print(f"Average BLEU Score: {average_bleu * 100:.2f}")
  # Assuming 'reference_summary' and 'hypothesis_summary' are Pandas Series
  references = dataset['headline'].astype(str).tolist()
  hypotheses = dataset[model_summaries].astype(str).tolist()

# Create a RougeScorer instance
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

# Calculate ROUGE scores for each pair of reference and hypothesis
  rouge_scores = []
  for ref, hyp in zip(references, hypotheses):
    scores = scorer.score(ref, hyp)
    rouge_scores.append(scores)

# Print the scores for the first pair (adjust as needed)
  print("ROUGE-1 F1 Score:", rouge_scores[0]['rouge1'].fmeasure)
  print("ROUGE-2 F1 Score:", rouge_scores[0]['rouge2'].fmeasure)
  print("ROUGE-L F1 Score:", rouge_scores[0]['rougeLsum'].fmeasure)


In [11]:
prediction = pd.read_pickle('/content/drive/MyDrive/wikihow_data/predictions_all.pkl')

### **4. Making Predictions**

#### **LSTM**

In [ ]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
LSTM_model = tf.keras.models.load_model("/content/drive/MyDrive/Models/wikihow-lstm.h5")

In [ ]:
tokenizer_doc = Tokenizer()
tokenizer_doc.fit_on_texts(X_test)
sequences_doc = tokenizer_doc.texts_to_sequences(X_test)

tokenizer_sum = Tokenizer()
tokenizer_sum.fit_on_texts(y_test)
sequences_sum = tokenizer_sum.texts_to_sequences(y_test)

X_test_padded = pad_sequences(sequences_doc, maxlen=2667, padding='post')
y_test_padded = pad_sequences(sequences_sum, maxlen=200, padding='post')

In [ ]:
import numpy as np
batch_size = 8
num_batches = len(X_test_padded) // batch_size

y_pred_texts = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    # Extract a batch of input sequences
    batch_padded_doc = X_test_padded[start_idx:end_idx]

    # Generate predictions for the batch
    batch_predicted_summary_sequences = LSTM_model.predict(batch_padded_doc)

    # Get word indices
    batch_word_indices = np.argmax(batch_predicted_summary_sequences, axis=-1)

    # Convert indices to words
    batch_y_pred_texts = [tokenizer_sum.sequences_to_texts([seq])[0] for seq in batch_word_indices]

    # Extend the result list
    y_pred_texts.extend(batch_y_pred_texts)

# If there's a remainder (i.e., len(padded_doc) is not a multiple of batch_size), process the remaining samples
if len(X_test_padded) % batch_size != 0:
    remainder_start_idx = num_batches * batch_size
    remainder_padded_doc = X_test_padded[remainder_start_idx:]

    remainder_predicted_summary_sequences = LSTM_model.predict(remainder_padded_doc)
    remainder_word_indices = np.argmax(remainder_predicted_summary_sequences, axis=-1)
    remainder_y_pred_texts = [tokenizer_sum.sequences_to_texts([seq])[0] for seq in remainder_word_indices]

    y_pred_texts.extend(remainder_y_pred_texts)

1/1 [==============================] - 0s 82ms/step


In [ ]:
prediction['LSTM_summary'] = y_pred_texts

In [ ]:
prediction.to_pickle('/content/drive/MyDrive/wikihow_data/predictions_all.pkl')

#### **T5 Small/BART Base(Without Finetuning)**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load your pre-trained model and tokenizer
#checkpoint = "t5-small"
checkpoint = "facebook/bart-base"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
sampled_df = dataset

# Specify the batch size
batch_size = 20

# Tokenize the sampled dataset with a progress bar
tokenized_batches = []

num_batches = int(np.ceil(len(sampled_df) / batch_size))

for i in tqdm(range(0, len(sampled_df), batch_size), total=num_batches, desc="Tokenizing"):
    # Extract a batch of articles from the sampled dataset
    batch_articles = sampled_df['text'][i:i + batch_size].tolist()


    #batch_articles = ["summarize: " + article for article in batch_articles] #Not Required for BART

    # Tokenize the batch with padding and truncation
    batch_tokenized = tokenizer(batch_articles, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

    # Check for inconsistent tensor sizes and filter them out
    valid_indices = [j for j in range(len(batch_articles)) if batch_tokenized['input_ids'][j].size(0) == 1024]

    # If all tensors have the expected size, append the batch
    if len(valid_indices) == len(batch_articles):
        tokenized_batches.append(batch_tokenized)
    else:
        print(f"Excluding batch at index {i} due to inconsistent tensor sizes.")

# Concatenate the valid batches
tokenized_dataset = {}
for key in tokenized_batches[0].keys():
    tokenized_dataset[key] = torch.cat([batch[key] for batch in tokenized_batches], dim=0)

Tokenizing: 100%|██████████| 279/279 [00:13<00:00, 21.29it/s]


In [ ]:
# Move the model to the GPU
model = model.to("cuda")

# Define batch size for inference
batch_size = 20

# Initialize an empty list to store decoded summaries
decoded_summaries = []
processed_indices = []

# Use tqdm to create a progress bar
for i in tqdm(range(0, len(tokenized_dataset["input_ids"]), batch_size), desc="Generating Summaries", leave=False):
    batch_inputs = {key: value[i:i + batch_size].to('cuda') for key, value in tokenized_dataset.items()}

    # Perform inference on the batch using the GPU
    with torch.no_grad():
        batch_outputs = model.generate(**batch_inputs)

    # Decode the generated summaries for the batch
    batch_decoded_summaries = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)
    decoded_summaries.extend(batch_decoded_summaries)

    # Keep track of the original indices
    batch_indices = sampled_df.index[i:i + batch_size].tolist()
    processed_indices.extend(batch_indices)


In [ ]:
if len(processed_indices) == len(decoded_summaries):
    # Add the decoded summaries to your DataFrame using the processed indices
    sampled_df.loc[processed_indices, "BART_summaries"] = decoded_summaries
else:
    print("Error: Lengths do not match.")

print(len(sampled_df))
print(len(processed_indices))
print(len(decoded_summaries))

5577
5577
5577


In [ ]:
sampled_df.head()

,text,headline,BART_summaries
0,"Caged animals such as hamsters, guinea pigs, r...",Relocate your caged pet.\nChoose an environmen...,"Caged animals such as hamsters, guinea pigs, r..."
1,Every boarding barn has people who go on vacat...,Horse-sit.\nHire yourself out as a groomer.\nT...,Every boarding barn has people who go on vacat...
2,Let him know that you like him by making an ef...,Pay attention to your appearance.\nShow him ho...,Let him know that you like him by making an ef...
3,It can be hard to tell if someone is intereste...,Read the signs.\nInvite him along on a group a...,It can be hard to tell if someone is intereste...
4,Your crush may be the prettiest person in the ...,Get to know your crush a bit.\nHang out in per...,Your crush may be the prettiest person in the ...


In [ ]:
import numpy as np

# Define a filtering criterion
def is_valid_summary(summary):
    return isinstance(summary, str) and summary.strip() != '' and len(set(summary)) > 1

# Apply the filter to create a subset of valid summaries
valid_summaries_df = sampled_df[sampled_df['BART_summaries'].apply(is_valid_summary)]

In [ ]:
prediction['BART_summary'] = valid_summaries_df['BART_summaries']

In [ ]:
prediction['T5_summary'] = valid_summaries_df['T5_summary']

In [ ]:
prediction.to_pickle('/content/drive/MyDrive/wikihow_data/predictions_all.pkl')

#### **T5 Simple FineTuning**

In [ ]:
from transformers import pipeline

t5simpleFT = pipeline("summarization", model="Prashanth-1998/wikihow_t5small_model", device=device)

In [ ]:
tqdm.pandas(desc="Applying summarizer")
prediction['T5_SFT_summary'] = dataset['text'].progress_apply(t5simpleFT)

Applying summarizer:   2%|▏         | 11/600 [00:11<07:21,  1.33it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Applying summarizer: 100%|██████████| 600/600 [08:43<00:00,  1.15it/s]


In [ ]:
prediction.to_csv('/content/drive/MyDrive/wikihow_data/predictions.csv', index=False)

#### **T5 LoRA Finetuning**

In [ ]:
t5LoRAFT = pipeline("summarization", model="rohitmacherla3/wikihow_t5small_LoRA_fine_tuned",device = device)

In [ ]:
tqdm.pandas(desc="Applying summarizer")
prediction['T5_LoRA_summary'] = dataset['text'].progress_apply(t5LoRAFT)

Applying summarizer: 100%|██████████| 5577/5577 [1:06:40<00:00,  1.39it/s]


In [ ]:
prediction.to_pickle('/content/drive/MyDrive/wikihow_data/predictions_all.pkl')

### **Predictions Review**

In [26]:
prediction.head()

,text,headline,Transformer_summary,T5_SFT_summary,T5_LoRA_summary,BART_summary,T5_summary,T5_Prompt_summary,LSTM_summary
0,"Caged animals such as hamsters, guinea pigs, r...",Relocate your caged pet.\nChoose an environmen...,know risks,[{'summary_text': 'a kennel should be the last...,[{'summary_text': 'Caged animals can be taken ...,"Caged animals such as hamsters, guinea pigs, r...",. Make sure the kennel you choose caters to yo...,a kennel should be the last resort for this ty...,consider the the the the the the the the the t...
1,Every boarding barn has people who go on vacat...,Horse-sit.\nHire yourself out as a groomer.\nT...,walk around park walk around park walk around ...,[{'summary_text': 'boarding your horse in a sh...,[{'summary_text': 'boarding barns offer to pul...,Every boarding barn has people who go on vacat...,",,,,,,,,,,,,,,,",boarding barns are a great way to earn income....,consider the the the the the the the the the t...
2,Let him know that you like him by making an ef...,Pay attention to your appearance.\nShow him ho...,look like girls girls girls girls girls girls ...,"[{'summary_text': 'if you're talking to a guy,...",[{'summary_text': 'you can flirt subtly and wo...,Let him know that you like him by making an ef...,", but don't be afraid to be a little sexy. Let",a guy can be a bit awkward when flirting with ...,consider the the the the the the the the the t...
3,It can be hard to tell if someone is intereste...,Read the signs.\nInvite him along on a group a...,talk friend,"[{'summary_text': 'if a person is interested, ...","[{'summary_text': 'some schools hold a ""sadie ...",It can be hard to tell if someone is intereste...,"you want him to hang out with, and have a spec...",if you want to get to know him before you ask ...,consider the the the the the the the the the t...
4,Your crush may be the prettiest person in the ...,Get to know your crush a bit.\nHang out in per...,make sure person want make sure person want ma...,"[{'summary_text': 'if you’re into a girl, tell...",[{'summary_text': 'sally kohn: texting your cr...,Your crush may be the prettiest person in the ...,", and you can tell her that she loves you. you...","if you're a crush, make sure you like her pers...",consider the the the the the the the the the t...


In [27]:
prediction['headline']

0       Relocate your caged pet.\nChoose an environmen...
1       Horse-sit.\nHire yourself out as a groomer.\nT...
2       Pay attention to your appearance.\nShow him ho...
3       Read the signs.\nInvite him along on a group a...
4       Get to know your crush a bit.\nHang out in per...
                              ...                        
5572    Find someone you feel comfortable with to go o...
5573    Double check your harness, bar or other restra...
5574    Learn the 20 classifications the Library of Co...
5575    Learn the difference between brake pads and br...
5576    Gather your materials.\nPrepare your yarn.\nAr...
Name: headline, Length: 5577, dtype: object

In [28]:
prediction['LSTM_summary']

0       consider the the the the the the the the the t...
1       consider the the the the the the the the the t...
2       consider the the the the the the the the the t...
3       consider the the the the the the the the the t...
4       consider the the the the the the the the the t...
                              ...                        
5572    consider the the the the the the the the the t...
5573    consider the the the the the the the the the t...
5574    consider the the the the the the the the the t...
5575    consider the the the the the the the the the t...
5576    consider the the the the the the the the the t...
Name: LSTM_summary, Length: 5577, dtype: object

In [29]:
prediction['Transformer_summary']

0                                              know risks
1       walk around park walk around park walk around ...
2       look like girls girls girls girls girls girls ...
3                                             talk friend
4       make sure person want make sure person want ma...
                              ...                        
5572                                   call driver ticket
5573    place cigarette hold cigarette hold cigarette ...
5574                                     look handwriting
5575    look signs injury look signs injury look signs...
5576    gather supplies place place place place place ...
Name: Transformer_summary, Length: 5577, dtype: object

In [30]:
prediction['BART_summary']

0       Caged animals such as hamsters, guinea pigs, r...
1       Every boarding barn has people who go on vacat...
2       Let him know that you like him by making an ef...
3       It can be hard to tell if someone is intereste...
4       Your crush may be the prettiest person in the ...
                              ...                        
5572    ; Also, If you're having trouble with the seat...
5573    Listen for the click or other sound it makes w...
5574    Each class corresponds to a letter of the alph...
5575    Brake pads and brake shoes are different from ...
5576    You will need to purchase either a 1/2 inch or...
Name: BART_summary, Length: 5577, dtype: object

In [31]:
prediction['T5_summary']

0       . Make sure the kennel you choose caters to yo...
1                                         ,,,,,,,,,,,,,,,
2          , but don't be afraid to be a little sexy. Let
3       you want him to hang out with, and have a spec...
4       , and you can tell her that she loves you. you...
                              ...                        
5572    you feel like you need to get out of line for ...
5573    the ride. You can also relax and relax. You ca...
5574    , the more specific the classification is—and ...
5575    , and brake shoes are different from one anoth...
5576    yarn. Cut each strand at one continuous angle,...
Name: T5_summary, Length: 5577, dtype: object

In [32]:
prediction['T5_Prompt_summary']

0       a kennel should be the last resort for this ty...
1       boarding barns are a great way to earn income....
2       a guy can be a bit awkward when flirting with ...
3       if you want to get to know him before you ask ...
4       if you're a crush, make sure you like her pers...
                              ...                        
5572    a ride like this will shake loose hats, sandal...
5573    'clicking' restraints are still safe and won't...
5574    the more numbers and letters contained in a ca...
5575    brake pads and brake shoes are different from ...
5576    you will need to purchase either a 1/2 inch or...
Name: T5_Prompt_summary, Length: 5577, dtype: object

In [33]:
prediction['T5_SFT_summary']

0       [{'summary_text': 'a kennel should be the last...
1       [{'summary_text': 'boarding your horse in a sh...
2       [{'summary_text': 'if you're talking to a guy,...
3       [{'summary_text': 'if a person is interested, ...
4       [{'summary_text': 'if you’re into a girl, tell...
                              ...                        
5572    [{'summary_text': 'a ride like this will shake...
5573    [{'summary_text': ''clicking' restraints are s...
5574    [{'summary_text': 'the more numbers and letter...
5575    [{'summary_text': 'the brake pad is built for ...
5576    [{'summary_text': 'the length of the rod depen...
Name: T5_SFT_summary, Length: 5577, dtype: object

In [34]:
prediction['T5_LoRA_summary']

0       [{'summary_text': 'Caged animals can be taken ...
1       [{'summary_text': 'boarding barns offer to pul...
2       [{'summary_text': 'you can flirt subtly and wo...
3       [{'summary_text': 'some schools hold a "sadie ...
4       [{'summary_text': 'sally kohn: texting your cr...
                              ...                        
5572    [{'summary_text': 'a ride like this will shake...
5573    [{'summary_text': ''clicking' restraints are s...
5574    [{'summary_text': 'the more numbers and letter...
5575    [{'summary_text': 'the brake pad is built for ...
5576    [{'summary_text': 'the length of the rod depen...
Name: T5_LoRA_summary, Length: 5577, dtype: object

### **Evaluating the Models using ROUGE and BLEU Scores**

**LSTM**

In [12]:
getmetrics(prediction,'LSTM_summary')

Average BLEU Score: 0.00
ROUGE-1 F1 Score: 0.0
ROUGE-2 F1 Score: 0.0
ROUGE-L F1 Score: 0.0


**Seq2Seq Transformer**

In [13]:
getmetrics(prediction,'Transformer_summary')

Average BLEU Score: 0.00
ROUGE-1 F1 Score: 0.0
ROUGE-2 F1 Score: 0.0
ROUGE-L F1 Score: 0.0


**BART-base Pre-trained**

In [25]:
getmetrics(prediction,'BART_summary')

Average BLEU Score: 0.09
ROUGE-1 F1 Score: 0.10526315789473682
ROUGE-2 F1 Score: 0.0
ROUGE-L F1 Score: 0.10526315789473682


**T5-Small Pre-trained**

In [15]:
getmetrics(prediction,'T5_summary')

Average BLEU Score: 0.11
ROUGE-1 F1 Score: 0.3
ROUGE-2 F1 Score: 0.10526315789473684
ROUGE-L F1 Score: 0.25


**T5-Small Pre-trained With prompt**

In [20]:
getmetrics(prediction,'T5_Prompt_summary')

Average BLEU Score: 0.10
ROUGE-1 F1 Score: 0.2
ROUGE-2 F1 Score: 0.05263157894736842
ROUGE-L F1 Score: 0.2


**T5-Small Simple Fine-tuned**

In [17]:
getmetrics(prediction,'T5_SFT_summary')

Average BLEU Score: 0.37
ROUGE-1 F1 Score: 0.2318840579710145
ROUGE-2 F1 Score: 0.05970149253731344
ROUGE-L F1 Score: 0.17391304347826086


**T5-small LoRA Fine-tuned**

In [ ]:
getmetrics(prediction,'T5_LoRA_summary')

Average BLEU Score: 0.34
ROUGE-1 F1 Score: 0.33333333333333337
ROUGE-2 F1 Score: 0.0625
ROUGE-L F1 Score: 0.18181818181818182
